In [269]:
"""
snipper.io/s/-MSTCnlcnC3p7aUSnOm_
https://www.kaggle.com/c/clickbait-news-detection/

"""

'\nsnipper.io/s/-MSTCnlcnC3p7aUSnOm_\nhttps://www.kaggle.com/c/clickbait-news-detection/\n\n'

In [270]:
sc

<SparkContext master=local[2] appName=clickbait>

In [271]:
from copy import deepcopy

from pyspark.ml.feature import Tokenizer, StopWordsRemover, NGram, HashingTF, IDF, CountVectorizer, StringIndexer #Word2Vec, OneHotEncoder, RegexTokenizer,
from pyspark.sql.functions import col, udf, length, avg, lit, concat, size, array, countDistinct
from pyspark.sql.types import IntegerType, StringType, ArrayType
#from pyspark.mllib.classification import NaiveBayes#, NaiveBayesModel
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.sql import SparkSession

import re
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
#import nltk
#nltk.download('wordnet')

In [272]:
try:
    sc.stop()
except:
    print("no sc to stop")
    
# try:
#     spark.stop()
# except Exception as e:
#     print("{}, {}".format(e,type(e)))
    

In [273]:
conf = SparkConf().setAppName('clickbait').setMaster('local[2]')
sc = SparkContext.getOrCreate(conf = conf)

In [274]:
spark = SparkSession.builder.appName('ClickBait')\
    .config("spark.driver.memory", "8g")\
    .config("spark.executor.memory", "2g")\
    .getOrCreate()

In [275]:
spark

In [276]:
train_path = "file:///home/hadoop/data/clickbait/train.csv"
#train_path = "/data/train.csv"
#train_path = "hdfs://localhost:9000/data/train.csv;"


In [277]:
# load main training dataframe

t_df = spark.read.csv(train_path, header=True, inferSchema=True, encoding="utf-8")

t_df=t_df.filter((t_df.label == "news")|(t_df.label == "clickbait"))

t_df.persist()

DataFrame[id: string, title: string, text: string, label: string]

In [278]:
t_df.select("label").distinct().show(), t_df.count()

+---------+
|    label|
+---------+
|     news|
|clickbait|
+---------+



(None, 11875)

In [279]:
t_df.filter((t_df.label == "news")).count(),t_df.filter((t_df.label == "clickbait")).count(),

(9371, 2504)

In [280]:
t_df=t_df.na.drop()

t_df.persist()

DataFrame[id: string, title: string, text: string, label: string]

In [281]:
t_df.count()

11807

In [282]:
news_df = t_df.select("*").where(col('label')=="news")
cb_df = t_df.select("*").where(col('label')=="clickbait")

In [283]:
news_df.count(), cb_df.count()

(9327, 2480)

In [284]:
news_seed=4
news_df=news_df.sample(withReplacement=None, fraction=1.0, seed=news_seed).select("*").limit(int(cb_df.count())) # seed selection -9

In [285]:
news_df=news_df.sample(withReplacement=None, fraction=1.0, seed=news_seed).select("*").limit(1600) # seed selection -9
cb_df=cb_df.sample(withReplacement=None, fraction=1.0, seed=news_seed).select("*").limit(1600) # seed selection -9

In [286]:
news_df.count(), cb_df.count()


(1600, 1600)

In [287]:
t_df = news_df.union(cb_df)
t_df.persist()
t_df.count()


3200

In [288]:
#from string import punctuation

def strip_possessive(l_):
    ptrn=re.compile(r"['`’]s", flags=re.I)
    r__ = [ptrn.sub('', s_) for s_ in l_]
    return(r__)  

@udf("string")
def strip_possessive_str(s_):
    ptrn=re.compile(r"['`’]s", flags=re.I)
    r__ = ptrn.sub(' ', s_)
    return(r__)


###
def strip_func(l_):
    ptrn = re.compile('[\W_]+')
    r__ = [ptrn.sub('', s_) for s_ in l_]
    return(r__)

@udf("string")
def strip_func_str(s_):
    ptrn = re.compile('[\W_]+')
    r__ = ptrn.sub(' ', s_)
    return(r__)

#udf_strip_func = udf(strip_func, ArrayType(StringType()))

# def strip_pos_spchars(l_):
#     ptrn_pos = re.compile(r"[']s", flags=re.I)
#     ptrn_spchars = re.compile('[\W_]+')
#     r__ = [ptrn.sub('', s_) for s_ in l_]
#     [ptrn_spchars.sub('', ptrn_pos.sub('', s_)) for s_ in l_]
#     return(r__)     
#x = list(one_sss_list.take(1)[0].asDict()['sss'])

#x = ["World's fair", "Biden's administration", "fergi's song", "Xi's policy"]

#x, strip_func(x), strip_possessive(x), #strip_possessive_str("World's fair ... Biden's administration, Fergi's song. Xi's policy")
#strip_func(one_sss_list.take(1)[0].asDict()['sss'])

In [289]:
help(udf)

Help on function udf in module pyspark.sql.functions:

udf(f=None, returnType=StringType)
    Creates a user defined function (UDF).
    
    .. note:: The user-defined functions are considered deterministic by default. Due to
        optimization, duplicate invocations may be eliminated or the function may even be invoked
        more times than it is present in the query. If your function is not deterministic, call
        `asNondeterministic` on the user defined function. E.g.:
    
    >>> from pyspark.sql.types import IntegerType
    >>> import random
    >>> random_udf = udf(lambda: int(random.random() * 100), IntegerType()).asNondeterministic()
    
    .. note:: The user-defined functions do not support conditional expressions or short circuiting
        in boolean expressions and it ends up with being executed all internally. If the functions
        can fail on special rows, the workaround is to incorporate the condition into the functions.
    
    .. note:: The user-defined

In [290]:
stemmer=SnowballStemmer("english")

lemmatizer = WordNetLemmatizer()


def stem_lem_per_word(word, pos='n'):
    sl_word = stemmer.stem(word)
    sl_word = lemmatizer.lemmatize(sl_word, pos=pos)
    return(sl_word)
    #pos=noun
    

def stem_lem_list(list_of_words, pos='n'):
    # l_ is supposed to be a  column value but nltk stemmers are expecting just a python list?
    r_list = [stem_lem_per_word(w_) for w_ in list_of_words]
    r_list = [w_ for w_ in r_list if w_ not in [""]]
    return(r_list)

@udf("array<string>")
def stem_lem(arr_str):
#    pass
     return stem_lem_list(arr_str)



st_cases=["writing", "write", "wrote", "writing", "written", "drag", "dragged", "drug", "dragon", ""]
[stem_lem_per_word(case) for case in st_cases]
#t_df.select("s3").map(stem_lem)
stem_lem_list(st_cases)


['write',
 'write',
 'wrote',
 'write',
 'written',
 'drag',
 'drag',
 'drug',
 'dragon']

In [291]:

# def stem_lem(w_, pos='n'):
#     return(lemmatizer.lemmatize(stemmer.stem(w_), pos=pos))

# stem_lem_cases= ["writing", "write", "wrote", "writing", "written", "drag", "dragged", "drug", "dragon"]

# display(
#     stem_lem_cases,
#     [stemmer.stem(w) for w in stem_lem_cases],
#     [lemmatizer.lemmatize(w, pos='n') for w in stem_lem_cases],
#     [stem_lem(w) for w in stem_lem_cases]
#     )

#stem_lem = udf(stem_lem, StringType())

In [292]:
t_df.columns, t_df

(['id', 'title', 'text', 'label'],
 DataFrame[id: string, title: string, text: string, label: string])

In [293]:
t_df.select('title','text','label').limit(20).show()


+--------------------+--------------------+-----+
|               title|                text|label|
+--------------------+--------------------+-----+
|China and Economi...|Economists genera...| news|
|Trade to Be a Big...|LONDON—British Pr...| news|
|Surgeon claiming ...|An Italian neuros...| news|
|This Is How Diffe...|Spanx does not do...| news|
|Samantha Bee and ...|“Jane Pauley! You...| news|
|$94.4m $198m $112...|In the very best ...| news|
|The fabulous life...|Madeline Stone un...| news|
|President Donald ...|The Super Bowl ch...| news|
|Freddie Flintoff ...|Stephen Manderson...| news|
|Air Force Airlift...|In a much needed ...| news|
|EXCLUSIVE: 'I'm n...|The man volunteer...| news|
|Bill Seeks to Enl...|Though she had pr...| news|
|Watch E! News Now...| Deck the halls,...| news|
|Joe Scarborough: ...|First lady Melani...| news|
|ECB Stimulus Disa...|European Central ...| news|
|Tweak It or Junk ...|President Donald ...| news|
|Detroit’s Resurge...|DETROIT—Concerns ...| news|


In [294]:
t_df=t_df.withColumn("s", 
                strip_func_str(strip_possessive_str(
                    concat(col('title'),col("text")))
                              ))

In [295]:
#t_df.select('title','text','label').limit(20).show()
t_df['text']#.limit(20).show()


Column<b'text'>

In [296]:
t_df = t_df.withColumn("s", strip_func_str(strip_possessive_str(
    concat(col('title'),col("text")))))

In [297]:
#t_df = t_df.withColumn("s", strip_func_str(strip_possessive_str(col('s'))))


t_df.select('s').show(truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [298]:
concat, col

(<function pyspark.sql.functions.concat(*cols)>,
 <function pyspark.sql.functions._create_function.<locals>._(col)>)

In [299]:
t_df.columns

['id', 'title', 'text', 'label', 's']

In [300]:
"’"=="`"

False

In [301]:
selection = ['s', "title", "text"]

display("avg string length of columns")

t_df \
    .select(*(length(col(c)).alias(c) for c in selection)) \
    .agg(*(avg(col(c)).alias(c) for c in selection)).show()

# t_df \
#     .select(*(size(col(c)).alias(c) for c in selection)) \
#     .agg(*(avg(col(c)).alias(c) for c in selection)).show()


'avg string length of columns'

+-----------+-------+---------+
|          s|  title|     text|
+-----------+-------+---------+
|3499.689375|73.2125|3539.1875|
+-----------+-------+---------+



In [302]:
from collections.abc import Iterable

def get_avg_len_cols(df, selection):
    """for str length in pyspark columns"""
    assert isinstance(selection, Iterable)
    return(
        df \
        .select(*(length(col(c)).alias(c) for c in selection)) \
        .agg(*(avg(col(c)).alias(c) for c in selection)).show()
              )

def get_avg_size_cols(df, selection):
    """for arrays of str in pyspark df"""
    assert isinstance(selection, Iterable)
    return(
        df \
        .select(*(size(col(c)).alias(c) for c in selection)) \
        .agg(*(avg(col(c)).alias(c) for c in selection)).show()
              )

In [303]:
t_df.columns
#get_avg_size_cols

['id', 'title', 'text', 'label', 's']

In [304]:
get_avg_len_cols(t_df, ["title", "text", "s"])

+-------+---------+-----------+
|  title|     text|          s|
+-------+---------+-----------+
|73.2125|3539.1875|3499.689375|
+-------+---------+-----------+



In [305]:
t_df = Tokenizer(inputCol = "s", outputCol = "s1").transform(t_df) # first text transformation is pyspark builtin tokenizer
#some punctuation like ',' is still showing after Tokenizer().transform. ??

In [306]:
t_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)
 |-- s: string (nullable = true)
 |-- s1: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [307]:
t_df.select("s1").limit(5).show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [308]:
t_df = StopWordsRemover(inputCol = "s1", outputCol="s2").transform(t_df)



In [309]:
get_avg_size_cols(t_df, ['s1','s2'])

+-----------+-----------+
|         s1|         s2|
+-----------+-----------+
|616.2259375|359.0196875|
+-----------+-----------+



In [310]:
# get_avg_size_cols(t_df, ['sss'])

In [311]:
# t_df.select('sss').limit(5).show(truncate=False)

In [312]:
# t_df.sample(False, 0.1, seed=0).limit(1).sss.getItem()

In [313]:
t_df.sample(False, 0.1, seed=0).select("s2").limit(1).show(truncate=False)

#my_sample=t_df.sample(False, 0.1, seed=0).select("s2").limit(1)


+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|s2                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
+-----------------------------------------------------------------------------

In [314]:
#
# get_avg_size_cols(t_df, ["s2"])

#Array(my_sample)



In [315]:
# t_df.select("s2").show()



In [316]:
t_df=t_df.withColumn("s3",stem_lem("s2"))

In [317]:
t_df.select("s3").take(3)



[Row(s3=['china', 'econom', 'reform', 'xi', 'jinp', 'track', 'record', 'economist', 'general', 'agre', 'china', 'must', 'overhaul', 'huge', 'wast', 'economi', 'want', 'continu', 'grow', 'year', 'come', 'mean', 'limit', 'polit', 'interfer', 'bank', 'financi', 'system', 'make', 'bloat', 'industri', 'sensit', 'market', 'forc', 'lower', 'barrier', 'foreign', 'trade', 'invest', 'propon', 'took', 'heart', 'late', '2012', 'presid', 'xi', 'jinp', 'took', 'formal', 'control', 'chine', 'communist', 'parti', 'pledg', 'crack', 'corrupt', 'cut', 'red', 'tape', 'today', 'mr', 'xi', 'near', 'end', 'first', 'five', 'year', 'term', 'much', 'optim', 'among', 'economist', 'fade', 'china', 'remain', 'heavili', 'depend', 'larg', 'invest', 'maintain', 'brisk', 'slow', 'econom', 'growth', 'steep', 'rise', 'govern', 'steer', 'lend', 'still', 'administr', 'made', 'small', 'chang', 'hint', 'mr', 'xi', 'may', 'focus', 'econom', 'overhaul', 'start', 'second', 'term', 'mr', 'xi', 'done', 'far', 'import', 'done', '

In [318]:
t_df.select("s3")

"""
text = df.description[0]

# Create and generate a word cloud image:
wordcloud = WordCloud().generate(text)

# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()
"""

'\ntext = df.description[0]\n\n# Create and generate a word cloud image:\nwordcloud = WordCloud().generate(text)\n\n# Display the generated image:\nplt.imshow(wordcloud, interpolation=\'bilinear\')\nplt.axis("off")\nplt.show()\n'

In [319]:
# Typecast text into dataframe
# t_df.select("s3")

In [320]:
# rdd0 = t_df.select("s3").alias("s3").rdd.map(list)

In [321]:
# rdd0.take(1)#[0][0][0]

In [322]:
# rdd1=rdd0.flatMap(lambda word: (word, 1)) \
#              .reduceByKey(lambda a, b: a + b)


In [323]:
# rdd1.take(1)
# strip_func_str(strip_possessive_str(
#     concat(col('title'),col("text")
#           ))
               

#t_df = t_df.withColumn("s3", stem_lem(lit(col("s2"))))
# #t_df.persist()
# t_df.show("s3")

In [324]:
# t_df.select('sss', "ssss").limit(5).show(truncate=False)

In [325]:
from pyspark.ml import Pipeline

ngram_pipeline=Pipeline(stages=[NGram(n=i, inputCol="s3", outputCol= str(i)+"gram") for i in range(2,4)])
m = ngram_pipeline.fit(t_df)
t_df = m.transform(t_df)
t_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)
 |-- s: string (nullable = true)
 |-- s1: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- s2: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- s3: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 2gram: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 3gram: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [326]:
# NGram Creation
"""
plan would be to turn this into a Pipeline([Tokenizer,StopWordsRemover,Ngram1,Ngram2...])
"""
if 0:
    raise AssertionError('stop')


    # This should be a pipeline
    text_cols= ["s3"]
    for t_c in text_cols:
        for i in range(1,4):
            t_df= NGram(n=i, inputCol=t_c, outputCol=t_c+"_"+str(i)+"gram").transform(t_df)
    t_df.persist()

In [327]:
#t_df.select("s3_1gram", "s").show(truncate=True)

In [328]:
# show each token
# show me grams
# show me bigrams
# show me trigrams



In [329]:
# t_df.printSchema()

In [330]:
## Before getting into trigrams maybe we can count tokens.

In [331]:
# df = spark.createDataFrame(
#     [(0, ["a", "b a", "c"]), (1, ["a", "b", "b", "c", "a"])],
#     ["label", "raw"])
# cv = CountVectorizer(inputCol="raw", outputCol="vectors")
# model = cv.fit(df)
# model.transform(df).show(truncate=False)

In [332]:
# .
# : org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 34.0 failed 1 times, most recent failure: Lost task 0.0 in stage 34.0 (TID 22, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
# 	at java.util.Arrays.copyOf(Arrays.java:3236)
# 	at java.io.ByteArrayOutputStream.grow(ByteArrayOutputStream.java:118)
t_df.printSchema(), t_df.columns

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)
 |-- s: string (nullable = true)
 |-- s1: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- s2: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- s3: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 2gram: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 3gram: array (nullable = true)
 |    |-- element: string (containsNull = false)



(None,
 ['id', 'title', 'text', 'label', 's', 's1', 's2', 's3', '2gram', '3gram'])

In [333]:
#text_tokenizer = Tokenizer(inputCol = 'text', outputCol = 'text_words')
#ttl_tokenizer = Tokenizer(inputCol = 'title', outputCol = 'ttl_words')
#regex_tokenizer = RegexTokenizer(inputCol= 'sentence', outputCol= 'words', pattern='\\W')
#t_df = tokenizer.transform(t_df)
# t_df = text_tokenizer.transform(ttl_tokenizer.transform(t_df))
help(CountVectorizer)

Help on class CountVectorizer in module pyspark.ml.feature:

class CountVectorizer(pyspark.ml.wrapper.JavaEstimator, _CountVectorizerParams, pyspark.ml.util.JavaMLReadable, pyspark.ml.util.JavaMLWritable)
 |  Extracts a vocabulary from document collections and generates a :py:attr:`CountVectorizerModel`.
 |  
 |  >>> df = spark.createDataFrame(
 |  ...    [(0, ["a", "b", "c"]), (1, ["a", "b", "b", "c", "a"])],
 |  ...    ["label", "raw"])
 |  >>> cv = CountVectorizer(inputCol="raw", outputCol="vectors")
 |  >>> model = cv.fit(df)
 |  >>> model.transform(df).show(truncate=False)
 |  +-----+---------------+-------------------------+
 |  |label|raw            |vectors                  |
 |  +-----+---------------+-------------------------+
 |  |0    |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
 |  |1    |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
 |  +-----+---------------+-------------------------+
 |  ...
 |  >>> sorted(model.vocabulary) == ['a', 'b', 'c']
 |  True
 |  >>> countVectorize

In [334]:
## Feature building

# fcols = [
# #  'text_f_token',
#   'text_ngram_1',
#   'text_ngram_2',
#   'text_ngram_3',
# #  'title_f_token',
#   'title_ngram_1',
#   'title_ngram_2',
#   'title_ngram_3',
# ]



# cv = CountVectorizer(inputCol="text_ngram_2", outputCol="features")
# t_df=cv.fit(t_df).transform(t_df)




# ohe = OneHotEncoderEstimator(
#     inputCols= fcols,
#     outputCols=["feat_"+s for s in fcols])

#f_df=ohe.fit(t_df).transform(t_df)
    
#f_df=cv.fit(t_df).transform(t_df)
#OneHotEncoderEstimator
#hashing_tf = HashingTF(inputCol='words', outputCol='rawFeatures')

In [335]:
# idf= IDF(inputCol='rawFeatures', outputCol='features')
# idf_model = idf.fit(featurized_df)
# rescaled_df = idf_model.transform(featurized_df)

# rescaled_df.show(truncate=False)

t_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)
 |-- s: string (nullable = true)
 |-- s1: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- s2: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- s3: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 2gram: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 3gram: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [336]:
#hashingTF = HashingTF(inputCol="s3", outputCol="htf_features", numFeatures=2**11)


#t_df=hashingTF.transform(t_df)

#help(hashingTF)

In [337]:
#help(NaiveBayes)

In [338]:
indexer = StringIndexer(inputCol="label", outputCol="label_index")
t_df = indexer.fit(t_df).transform(t_df)
t_df.persist()


# ['id',
#  'title',
#  'text',
#  'label',
#  's',
#  'ss',
#  's1',
#  'sss',
#  's2',
#  's3',
#  's2_1gram',
#  's2_2gram',
#  's2_3gram',
#  's3_1gram',
#  's3_2gram',
#  's3_3gram',
#  's3_cv',
#  'label_index']

DataFrame[id: string, title: string, text: string, label: string, s: string, s1: array<string>, s2: array<string>, s3: array<string>, 2gram: array<string>, 3gram: array<string>, label_index: double]

In [339]:
# t_df.select('label','label_index').limit(2).show()

In [340]:
#t_df.drop()

In [341]:
PREPROCESSED_DF_CSV_PATH="file:///home/hadoop/cb/pre_train.csv"

#t_df.repartition(1).write.format('com.databricks.spark.csv').save(PREPROCESSED_DF_CSV_PATH,header = 'true')

#t_df.repartition(1).write.json(PREPROCESSED_DF_CSV_PATH,mode="overwrite")

t_df.schema

StructType(List(StructField(id,StringType,true),StructField(title,StringType,true),StructField(text,StringType,true),StructField(label,StringType,true),StructField(s,StringType,true),StructField(s1,ArrayType(StringType,true),true),StructField(s2,ArrayType(StringType,true),true),StructField(s3,ArrayType(StringType,true),true),StructField(2gram,ArrayType(StringType,false),true),StructField(3gram,ArrayType(StringType,false),true),StructField(label_index,DoubleType,false)))

In [342]:
t_df.columns

['id',
 'title',
 'text',
 'label',
 's',
 's1',
 's2',
 's3',
 '2gram',
 '3gram',
 'label_index']

In [343]:
#
if 0:

    c_v = CountVectorizer(inputCol="s3", outputCol="s3_cv",
                         minDF=2.0)

    # c_v = CountVectorizer(inputCol="3gram", outputCol="3gram_cv",
    #                      minDF=2.0, vocabSize=800)
    # c_v = CountVectorizer(inputCol="s3", outputCol="s3_cv",
    #                      minDF=2.0, vocabSize=800)

    #https://stackoverflow.com/questions/38839924/how-to-combine-n-grams-into-one-vocabulary-in-spark

    # _p = {'inputCol':"title_ngram_1", 'outputCol':"title_ngram_1_cv",
    #       'minDF':2.0, 'vocabSize':2**14}

    #m_ = c_v.fit(t_df, params={'inputCol': 'text_ngram_1'})

    mdl = c_v.fit(t_df)

    t_df=mdl.transform(t_df)
    t_df.persist()

In [344]:
# count_vects = [CountVectorizer(inputCol= i_, outputCol= i_+"_cv",
#                      minDF=2.0, vocabSize=800) for i_ in ["s3", '2gram', '3gram']]

In [345]:
columns_to_drop=[
#'id',
 'title',
 'text',
 'label',
 #'s',
 'ss',
 's1',
 'sss',
 's2',
# 's3',
 's2_1gram',
 's2_2gram',
#  's2_3gram',
#  's3_1gram',
#  's3_2gram',
#  's3_3gram',
#  's3_cv',
#  'label_index'
]

#t_df = t_df.drop(*columns_to_drop)

t_df.columns

['id',
 'title',
 'text',
 'label',
 's',
 's1',
 's2',
 's3',
 '2gram',
 '3gram',
 'label_index']

In [360]:
# 1-gram, 2-gram, 3-gram
gramcols=["s3", "2gram", "3gram"]
pipeline = Pipeline(stages=[CountVectorizer(inputCol=c__ , outputCol=c__+"_cv", minDF=2.0) for c__ in gramcols] +
                    [NaiveBayes(modelType= "multinomial", labelCol="label_index",
                                featuresCol = c__+"_cv",
                                rawPredictionCol="raw_pred_"+c__,
                                probabilityCol="prob_"+c__,
                                predictionCol="pred_"+c__) for c__ in gramcols]
                   )

In [361]:
#raise Exception("fail")

train, test = t_df.randomSplit([.8,.2], seed = 2) #news_seed 4, train/test seed 999, accuracy 1.0??



In [362]:
train.select("label").distinct().show(),
test.select("label").distinct().show()

+---------+
|    label|
+---------+
|     news|
|clickbait|
+---------+

+---------+
|    label|
+---------+
|     news|
|clickbait|
+---------+



In [349]:
news_seed

4

In [363]:
pipeline_model = pipeline.fit(train)
pred = pipeline_model.transform(test)


# pipeline_model = pipeline.fit(test)
# pred = pipeline_model.transform(train)

In [364]:
# NB = NaiveBayes(modelType= "multinomial", labelCol="label_index", featuresCol ="s3_cv")
#NB = NaiveBayes(modelType= "bernoulli", labelCol="label_index", featuresCol ="3gram_feats")

# nb_model= NB.fit(train)
# nb_predictions= nb_model.transform(test)


In [365]:
["pred_"+c__ for c__ in gramcols]

['pred_s3', 'pred_2gram', 'pred_3gram']

In [366]:

evaluator = MulticlassClassificationEvaluator(labelCol= "label_index", predictionCol = "pred_s3", 
                                              metricName="accuracy")
evaluator0 = MulticlassClassificationEvaluator(labelCol= "label_index", predictionCol = "pred_2gram", 
                                              metricName="accuracy")
evaluator1 = MulticlassClassificationEvaluator(labelCol= "label_index", predictionCol = "pred_3gram", 
                                              metricName="accuracy")

#nb_accuracy = evaluator.evaluate(nb_predictions)
nb_accuracy = evaluator.evaluate(pred)
nb_accuracy0 = evaluator0.evaluate(pred)
nb_accuracy1 = evaluator1.evaluate(pred)

print("1gram Accuracy : ", nb_accuracy)
print("2gram Accuracy : ", nb_accuracy0)
print("3gram Accuracy : ", nb_accuracy1)

#news seed 2: 0.68
#news seed 3: 0.68
#news seed 4: 0.68

1gram Accuracy :  0.6620370370370371
2gram Accuracy :  0.6975308641975309
3gram Accuracy :  0.6234567901234568


In [ ]:
"""
https://spark.apache.org/docs/2.2.0/ml-pipeline.html

Plan:
    - use Pipeline([transformer_n, estimator_n...])
    - Improve preprocessing (punctuation, special characters, stemming)
    - Include transformers and estimators for each n_gram length

"""

In [ ]:
ml_pipeline = Pipeline(stages=[NB,nb_predictions])

# Fit the pipeline to training documents.
model = pipeline.fit(training)
prediction = model.transform(test)